# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [1]:
import mysql.connector
from functools import partial

def get_connection_helper():
    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados2020',
        database='sakila'
    )
    return connection, partial(run_db_query, connection)

connection, db = get_connection_helper()

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Liste do maior para o menor.

In [2]:
db("SELECT category.name, SUM(payment.amount) FROM payment INNER JOIN rental USING(rental_id) INNER JOIN inventory USING(inventory_id) INNER JOIN film USING(film_id) INNER JOIN film_category USING(film_id) INNER JOIN category USING(category_id) GROUP BY category.category_id ORDER BY SUM(payment.amount) DESC")

Executando query:
('Sports', Decimal('5314.21'))
('Sci-Fi', Decimal('4756.98'))
('Animation', Decimal('4656.30'))
('Drama', Decimal('4587.39'))
('Comedy', Decimal('4383.58'))
('Action', Decimal('4375.85'))
('New', Decimal('4351.62'))
('Games', Decimal('4281.33'))
('Foreign', Decimal('4270.67'))
('Family', Decimal('4226.07'))
('Documentary', Decimal('4217.52'))
('Horror', Decimal('3722.54'))
('Children', Decimal('3655.55'))
('Classics', Decimal('3639.59'))
('Travel', Decimal('3549.64'))
('Music', Decimal('3417.72'))


Liste os filmes e o numero de vezes em que foram alugados

In [3]:
db("SELECT film.title, COUNT(rental.rental_id) FROM film INNER JOIN inventory USING(film_id) INNER JOIN rental USING(inventory_id) GROUP BY film.title ORDER BY COUNT(rental.rental_id) DESC LIMIT 10")

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('RIDGEMONT SUBMARINE', 32)
('GRIT CLOCKWORK', 32)
('SCALAWAG DUCK', 32)
('JUGGLER HARDLY', 32)
('FORWARD TEMPLE', 32)
('HOBBIT ALIEN', 31)
('ROBBERS JOON', 31)
('ZORRO ARK', 31)


## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir.

In [4]:
db('''
DROP VIEW IF EXISTS movie_count;
''')

db('''
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
''')

Executando query:
Executando query:


Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [5]:
db('''
SELECT * FROM movie_count LIMIT 30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [6]:
db("UPDATE film SET title = 'STONED PUNK' WHERE title LIKE 'DAZED PUNK'")

Executando query:


Verifique agora a nossa *view*:

In [7]:
db('''
SELECT * FROM movie_count LIMIT 30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


Como você pode ver, as views são tabelas virtuais que são automaticamente atualizadas quando as tabelas originais são modificadas.

In [8]:
connection.rollback()

## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [9]:
db('''
CREATE TEMPORARY TABLE long_film 
SELECT
    *
FROM
    film
WHERE
    film.length > 180;
''')

Executando query:


Podemos verificar que a tabela `long_film` agora existe:

In [10]:
db('DESCRIBE long_film')

Executando query:
('film_id', 'smallint unsigned', 'NO', '', '0', 'NULL')
('title', 'varchar(255)', 'NO', '', None, 'NULL')
('description', 'text', 'YES', '', None, 'NULL')
('release_year', 'year', 'YES', '', None, 'NULL')
('language_id', 'tinyint unsigned', 'NO', '', None, 'NULL')
('original_language_id', 'tinyint unsigned', 'YES', '', None, 'NULL')
('rental_duration', 'tinyint unsigned', 'NO', '', '3', 'NULL')
('rental_rate', 'decimal(4,2)', 'NO', '', '4.99', 'NULL')
('length', 'smallint unsigned', 'YES', '', None, 'NULL')
('replacement_cost', 'decimal(5,2)', 'NO', '', '19.99', 'NULL')
('rating', "enum('G','PG','PG-13','R','NC-17')", 'YES', '', 'G', 'NULL')
('special_features', "set('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')", 'YES', '', None, 'NULL')
('last_update', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', 'on update CURRENT_TIMESTAMP')


Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [11]:
db('SHOW TABLES')

Executando query:
('actor',)
('actor_info',)
('address',)
('category',)
('city',)
('country',)
('customer',)
('customer_list',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('film_text',)
('inventory',)
('language',)
('movie_count',)
('nicer_but_slower_film_list',)
('payment',)
('rental',)
('sales_by_film_category',)
('sales_by_store',)
('staff',)
('staff_list',)
('store',)


Vamos listar o conteudo desta tabela:

In [12]:
db('SELECT title FROM long_film')

Executando query:
('ANALYZE HOOSIERS',)
('BAKED CLEOPATRA',)
('CATCH AMISTAD',)
('CHICAGO NORTH',)
('CONSPIRACY SPIRIT',)
('CONTROL ANTHEM',)
('CRYSTAL BREAKING',)
('DARN FORRESTER',)
('FRONTIER CABIN',)
('GANGS PRIDE',)
('HAUNTING PIANIST',)
('HOME PITY',)
('HOTEL HAPPINESS',)
('INTRIGUE WORST',)
('JACKET FRISCO',)
('KING EVOLUTION',)
('LAWLESS VISION',)
('LOVE SUICIDES',)
('MONSOON CAUSE',)
('MOONWALKER FOOL',)
('MUSCLE BRIGHT',)
('POND SEATTLE',)
('RECORDS ZORRO',)
('REDS POCUS',)
('RUNAWAY TENENBAUMS',)
('SATURN NAME',)
('SCALAWAG DUCK',)
('SEARCHERS WAIT',)
('SMOOCHY CONTROL',)
('SOLDIERS EVOLUTION',)
('SONS INTERVIEW',)
('SORORITY QUEEN',)
('STAR OPERATION',)
('SWEET BROTHERHOOD',)
('THEORY MERMAID',)
('WIFE TURN',)
('WILD APOLLO',)
('WORST BANGER',)
('YOUNG LANGUAGE',)


Vamos apagar a tabela `long_film`:

In [13]:
db('DROP TABLE long_film')

Executando query:


### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria

In [14]:
db('''
DROP TEMPORARY TABLE IF EXISTS max_duration;
''')
db('CREATE TEMPORARY TABLE max_duration SELECT category.name as name, MAX(film.length) as max_len FROM film INNER JOIN film_category USING(film_id) INNER JOIN category USING(category_id) GROUP BY category.category_id')

Executando query:
Executando query:


 - Verifique a tabela.

In [15]:
db("SELECT * FROM max_duration")

Executando query:
('Action', 185)
('Animation', 185)
('Children', 178)
('Classics', 184)
('Comedy', 185)
('Documentary', 183)
('Drama', 181)
('Family', 184)
('Foreign', 184)
('Games', 185)
('Horror', 181)
('Music', 185)
('New', 183)
('Sci-Fi', 185)
('Sports', 184)
('Travel', 185)


- Agora use a tabela temporária para construir uma tabela com as categorias e seus respectivos filmes mais longos:

In [16]:
db("SELECT category.name, film.title FROM max_duration INNER JOIN category USING(name) INNER JOIN film_category USING(category_id) INNER JOIN film USING(film_id) WHERE film.length = max_duration.max_len GROUP BY category.name")

Executando query:
('Action', 'DARN FORRESTER')
('Animation', 'GANGS PRIDE')
('Children', 'FURY MURDER')
('Classics', 'CONSPIRACY SPIRIT')
('Comedy', 'CONTROL ANTHEM')
('Documentary', 'WIFE TURN')
('Drama', 'JACKET FRISCO')
('Family', 'KING EVOLUTION')
('Foreign', 'CRYSTAL BREAKING')
('Games', 'CHICAGO NORTH')
('Horror', 'ANALYZE HOOSIERS')
('Music', 'HOME PITY')
('New', 'FRONTIER CABIN')
('Sci-Fi', 'SOLDIERS EVOLUTION')
('Sports', 'SMOOCHY CONTROL')
('Travel', 'MUSCLE BRIGHT')


- delete a tabela temporária

In [17]:
db('''
DROP TEMPORARY TABLE IF EXISTS max_duration;
''')

Executando query:


### Desafio!

- Gere uma tabela contendo, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [18]:
# db("SELECT CONCAT(first_name, ' ', last_name), movs FROM (SELECT actor_id, actor.first_name, actor.last_name, CONCAT(category.name, ': ', GROUP_CONCAT(DISTINCT film.title)) AS movs FROM category INNER JOIN film_category USING(category_id) INNER JOIN film USING(film_id) INNER JOIN film_actor USING (film_id) INNER JOIN actor USING (actor_id)) AS partial GROUP BY actor_id")

db("SELECT actor_id, actor.first_name, actor.last_name, CONCAT(category.name, ': ', GROUP_CONCAT(DISTINCT film.title)) AS movs FROM category INNER JOIN film_category USING(category_id) INNER JOIN film USING(film_id) INNER JOIN film_actor USING (film_id) INNER JOIN actor USING (actor_id) WHERE first_name LIKE 'PENELOPE' AND last_name LIKE 'GUINESS' GROUP BY actor_id")

Executando query:
(1, 'PENELOPE', 'GUINESS', 'Documentary: ACADEMY DINOSAUR,ANACONDA CONFESSIONS,ANGELS LIFE,BULWORTH COMMANDMENTS,CHEAPER CLYDE,COLOR PHILADELPHIA,ELEPHANT TROJAN,GLEAMING JAWBREAKER,HUMAN GRAFFITI,KING EVOLUTION,LADY STAGE,LANGUAGE COWBOY,MULHOLLAND BEAST,OKLAHOMA JUMANJI,RULES HUMAN,SPLASH GUMP,VERTIGO NORTHWEST,WESTWARD SEABISCUIT,WIZARD COLDBLOODED')


## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [19]:
db('''
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
''')

Executando query:


Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [20]:
db('SELECT @avg_rate, @stddev_rate')

Executando query:
(Decimal('2.980000000'), 1.6455698101265719)


Agora podemos selecionar os filmes caros!

In [21]:
db('''
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate;
''')

Executando query:
('ACE GOLDFINGER', Decimal('4.99'))
('AIRPLANE SIERRA', Decimal('4.99'))
('AIRPORT POLLOCK', Decimal('4.99'))
('ALADDIN CALENDAR', Decimal('4.99'))
('ALI FOREVER', Decimal('4.99'))
('AMELIE HELLFIGHTERS', Decimal('4.99'))
('AMERICAN CIRCUS', Decimal('4.99'))
('ANTHEM LUKE', Decimal('4.99'))
('APACHE DIVINE', Decimal('4.99'))
('APOCALYPSE FLAMINGOS', Decimal('4.99'))
('ATTACKS HATE', Decimal('4.99'))
('ATTRACTION NEWTON', Decimal('4.99'))
('AUTUMN CROW', Decimal('4.99'))
('BABY HALL', Decimal('4.99'))
('BACKLASH UNDEFEATED', Decimal('4.99'))
('BEAST HUNCHBACK', Decimal('4.99'))
('BEAUTY GREASE', Decimal('4.99'))
('BEHAVIOR RUNAWAY', Decimal('4.99'))
('BETRAYED REAR', Decimal('4.99'))
('BIKINI BORROWERS', Decimal('4.99'))
('BILKO ANONYMOUS', Decimal('4.99'))
('BIRCH ANTITRUST', Decimal('4.99'))
('BIRD INDEPENDENCE', Decimal('4.99'))
('BIRDS PERDITION', Decimal('4.99'))
('BLINDNESS GUN', Decimal('4.99'))
('BOILED DARES', Decimal('4.99'))
('BOOGIE AMELIE', Decimal('4.99')

### Vamos praticar

Use variáveis temporárias para encontrar o ator que mais participou de filmes.

In [22]:
db("SELECT MAX(temp.cnt) INTO @max_count FROM (SELECT COUNT(film_id) AS cnt FROM film_actor GROUP BY actor_id) AS temp ")
db("SELECT CONCAT(first_name, ' ', last_name) FROM actor INNER JOIN film_actor USING (actor_id) GROUP BY actor_id HAVING COUNT(film_id) = @max_count")

Executando query:
Executando query:
('GINA DEGENERES',)


Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [23]:
db("SELECT * FROM movie_count WHERE title LIKE 'COWBOY DOOM'")

Executando query:
('COWBOY DOOM', 10)


Registre um aluguel do filme "COWBOY DOOM", feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'

In [24]:
db("SELECT film_id INTO @film_id FROM film WHERE title LIKE 'COWBOY DOOM'")
db("SELECT customer_id INTO @customer_id FROM customer WHERE first_name LIKE 'JESSIE' AND last_name LIKE 'BANKS'")
db("SELECT staff_id INTO @staff_id FROM staff WHERE first_name LIKE 'JON' AND last_name LIKE 'STEPHENS'")
db("INSERT INTO inventory (film_id, store_id, last_update) VALUES (@film_id, 1, CURRENT_TIME())") 
db("INSERT INTO rental (rental_date, inventory_id, customer_id, return_date, staff_id, last_update) VALUES ('2019-01-01', LAST_INSERT_ID(), @customer_id, '2019-01-08', @staff_id, CURRENT_TIME())")

Executando query:
Executando query:
Executando query:
Executando query:
Executando query:


Verifique usando a view que a contagem de alugueis do filme subiu.

In [25]:
db("SELECT title, cnt FROM movie_count WHERE title LIKE 'COWBOY DOOM'")

Executando query:
('COWBOY DOOM', 11)


## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [26]:
db('''
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
''')

Executando query:
(107, 'GINA', 'DEGENERES', 42)
(102, 'WALTER', 'TORN', 41)
(198, 'MARY', 'KEITEL', 40)


Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [27]:
db('DROP TABLE IF EXISTS temp_pop_actors')
db('''
CREATE TEMPORARY TABLE temp_pop_actors
SELECT first_name, last_name, actor_id FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    COUNT(film_id) DESC
LIMIT 3
''')
db('''
SELECT * from temp_pop_actors
''')

Executando query:
Executando query:
Executando query:
('GINA', 'DEGENERES', 107)
('WALTER', 'TORN', 102)
('MARY', 'KEITEL', 198)


Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [28]:
db('''
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
''')

Executando query:
('BED HIGHBALL',)
('CALENDAR GUNFIGHT',)
('CHAMBER ITALIAN',)
('CHAPLIN LICENSE',)
('CHARIOTS CONSPIRACY',)
('CLUELESS BUCKET',)
('COLDBLOODED DARLING',)
('CONEHEADS SMOOCHY',)
('DARKNESS WAR',)
('DEER VIRGINIAN',)
('DOGMA FAMILY',)
('ELEPHANT TROJAN',)
('EXCITEMENT EVE',)
('FRISCO FORREST',)
('GANDHI KWAI',)
('GOODFELLAS SALUTE',)
('GUNFIGHT MOON',)
('HALL CASSIDY',)
('HEARTBREAKERS BRIGHT',)
('HOOK CHARIOTS',)
('HYDE DOCTOR',)
('IMPACT ALADDIN',)
('INDIAN LOVE',)
('INTRIGUE WORST',)
('LICENSE WEEKEND',)
('LOUISIANA HARRY',)
('MAGNIFICENT CHITTY',)
('METAL ARMAGEDDON',)
('MIDNIGHT WESTWARD',)
('MOVIE SHAKESPEARE',)
('MUMMY CREATURES',)
('OPEN AFRICAN',)
('SEARCHERS WAIT',)
('SEVEN SWARM',)
('SIERRA DIVIDE',)
('SPIRITED CASUALTIES',)
('STORM HAPPINESS',)
('SUGAR WONKA',)
('TELEGRAPH VOYAGE',)
('TRAINSPOTTING STRANGERS',)
('WIFE TURN',)
('WINDOW SIDE',)
('AMELIE HELLFIGHTERS',)
('ARABIA DOGMA',)
('BANG KWAI',)
('CASABLANCA SUPER',)
('CASPER DRAGONFLY',)
('CROW GREASE',

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [29]:
db('DROP TABLE temp_pop_actors')

Executando query:


### Vamos praticar

Liste os atores que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora).

In [30]:
db('DROP TABLE IF EXISTS temp_top3_movies')
db("CREATE TEMPORARY TABLE temp_top3_movies SELECT film.film_id, SUM(payment.amount) FROM film INNER JOIN inventory USING (film_id) INNER JOIN rental USING (inventory_id) INNER JOIN payment USING (rental_id) WHERE payment_date IS NOT NULL GROUP BY film_id ORDER BY SUM(payment.amount) DESC LIMIT 3")
db("SELECT DISTINCT CONCAT(first_name, ' ', last_name) FROM actor INNER JOIN film_actor USING(actor_id) INNER JOIN film USING(film_id) WHERE film_id IN (SELECT film_id FROM temp_top3_movies)")
db('DROP TABLE temp_top3_movies')

Executando query:
Executando query:
Executando query:
('WOODY HOFFMAN',)
('CARMEN HUNT',)
('GINA DEGENERES',)
('LUCILLE DEE',)
('VIVIEN BASINGER',)
('MICHAEL BENING',)
('THORA TEMPLE',)
('JULIA BARRYMORE',)
('CAMERON ZELLWEGER',)
('GRETA MALDEN',)
('MICHAEL BOLGER',)
('IAN TANDY',)
('NICK DEGENERES',)
('LISA MONROE',)
Executando query:


## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, ás vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Vamos praticar

Vamos refazer a atividade dos filmes de atores populares, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

In [31]:
try:
    db('''
    SELECT 
        actor_id, first_name, last_name
    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    WHERE
        film_actor.film_id IN (
            SELECT 
                film_id
            FROM
                film
                INNER JOIN inventory USING (film_id)
                INNER JOIN rental USING (inventory_id)
                INNER JOIN payment USING (rental_id)
            WHERE
                payment_date IS NOT NULL
            GROUP BY film_id
            ORDER BY SUM(amount)
            LIMIT 3)
    ORDER BY actor_id
    ''')
except mysql.connector.ProgrammingError as e:
    print(f'ProgrammingError: {e}')

Executando query:
ProgrammingError: 1235 (42000): This version of MySQL doesn't yet support 'LIMIT & IN/ALL/ANY/SOME subquery'


Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

In [32]:
db('''
    SELECT 
        DISTINCT actor_id, first_name, last_name
    FROM
        (SELECT 
            film_id
        FROM
            film
            INNER JOIN inventory USING (film_id)
            INNER JOIN rental USING (inventory_id)
            INNER JOIN payment USING (rental_id)
        WHERE
            payment_date IS NOT NULL
        GROUP BY film_id
        ORDER BY SUM(amount) DESC
        LIMIT 3) AS top_movies
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
    ORDER BY actor_id
    ''')

Executando query:
(28, 'WOODY', 'HOFFMAN')
(47, 'JULIA', 'BARRYMORE')
(52, 'CARMEN', 'HUNT')
(107, 'GINA', 'DEGENERES')
(111, 'CAMERON', 'ZELLWEGER')
(138, 'LUCILLE', 'DEE')
(155, 'IAN', 'TANDY')
(157, 'GRETA', 'MALDEN')
(158, 'VIVIEN', 'BASINGER')
(166, 'NICK', 'DEGENERES')
(174, 'MICHAEL', 'BENING')
(178, 'LISA', 'MONROE')
(185, 'MICHAEL', 'BOLGER')
(200, 'THORA', 'TEMPLE')


# `UNION`

Quando duas tabelas tem **EXATAMENTE** as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador `UNION`. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [40]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('CREATE TEMPORARY TABLE nomes_clientes SELECT first_name, last_name FROM customer')

Executando query:
Executando query:


In [41]:
db('DESCRIBE nomes_clientes')
db('SELECT * FROM nomes_clientes LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')
('BARBARA', 'JONES')
('ELIZABETH', 'BROWN')


In [42]:
db('DROP TABLE IF EXISTS nomes_staff')
db('CREATE TEMPORARY TABLE nomes_staff SELECT first_name, last_name FROM staff')

Executando query:
Executando query:


In [43]:
db('DESCRIBE nomes_staff')
db('SELECT * FROM nomes_staff LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')


In [44]:
db('DROP TABLE IF EXISTS nomes_all')
db('CREATE TEMPORARY TABLE nomes_all (SELECT * FROM nomes_staff) UNION (SELECT * FROM nomes_clientes)')

Executando query:
Executando query:


In [45]:
db('DESCRIBE nomes_all')
db('SELECT * FROM nomes_all LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', '', 'NULL')
('last_name', 'varchar(45)', 'NO', '', '', 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')


In [46]:
db('DROP TABLE nomes_clientes')
db('DROP TABLE nomes_staff')
db('DROP TABLE nomes_all')

Executando query:
Executando query:
Executando query:


**Vamos praticar:** refaça o exemplo acima mas use *subqueries* ao invés de *temp tables*.

In [47]:
db("SELECT * FROM (SELECT first_name, last_name FROM staff UNION SELECT first_name, last_name FROM customer) AS all_names LIMIT 5")

Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')


## Desafios!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme!

In [86]:
db("SELECT film.title FROM film INNER JOIN film_actor USING (film_id) INNER JOIN actor USING (actor_id) WHERE SUBSTRING(film.title, 1, 1) LIKE SUBSTRING(actor.first_name, 1, 1) GROUP BY film.film_id HAVING COUNT(film.film_id) >= 2 ORDER BY film.title ASC")

Executando query:
('CAPER MOTIONS',)
('CLUELESS BUCKET',)
('CROOKED FROGMEN',)
('CROW GREASE',)
('CUPBOARD SINNERS',)
('EVOLUTION ALTER',)
('GABLES METROPOLIS',)
('JAPANESE RUN',)
('JAWBREAKER BROOKLYN',)
('JEDI BENEATH',)
('JUMANJI BLADE',)
('LESSON CLEOPATRA',)
('MADIGAN DORADO',)
('MAKER GABLES',)
('MALKOVICH PET',)
('MIXED DOORS',)
('MODEL FISH',)
('MUSCLE BRIGHT',)
('MUSIC BOONDOCK',)
('RANGE MOONWALKER',)
('SHAWSHANK BUBBLE',)
('SIEGE MADRE',)
('STAGECOACH ARMAGEDDON',)
('STREAK RIDGEMONT',)
('SUBMARINE BED',)
('TARZAN VIDEOTAPE',)
('WARDROBE PHANTOM',)
('WISDOM WORKER',)


Semana do "DAN HARRIS": liste os clientes que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas onde a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'

In [87]:
db('''
    SELECT
        COUNT(DISTINCT customer_id)
    FROM
        rental
        INNER JOIN inventory USING (inventory_id)
    WHERE
        customer_id IN
            (SELECT 
                DISTINCT customer_id
            FROM
                rental 
                INNER JOIN inventory USING (inventory_id)
                INNER JOIN film USING (film_id)
                INNER JOIN film_actor USING(film_id) 
                INNER JOIN actor USING (actor_id) 
            WHERE 
                actor.first_name LIKE 'DAN' 
            AND 
                actor.last_name LIKE 'HARRIS')
        AND 
            rental_date < STR_TO_DATE('2005-06-01', '%Y-%m-%d')
        OR
            customer_id NOT IN
                (SELECT 
                    DISTINCT customer_id
                FROM
                    rental 
                    INNER JOIN inventory USING (inventory_id)
                    INNER JOIN film USING (film_id)
                    INNER JOIN film_actor USING(film_id) 
                    INNER JOIN actor USING (actor_id) 
                WHERE 
                    actor.first_name LIKE 'DAN' 
                AND 
                    actor.last_name LIKE 'HARRIS')
''')

Executando query:
(557,)


## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso!

In [88]:
connection.close()